# Adapting the NFL 2021 Regular Season Simulator code
This notebook shows how to adapt the procedure used in the [NFL 2021 Regular Season Simulator](https://share.streamlit.io/christopherdavisuci/nfl2021-simulation/main/nfl_sim_app.py).  The adaptation we make is to add half a point to the offensive and defensive power ratings for a team coming off a bye.

You should click the "Duplicate" button above, (register for a free Deepnote account if you don't already have one), and try making your own adaptation.

I tried to include some examples of what the code is doing.  (If you look at the source code for the app mentioned above, it will be much harder to read.)

## Importing some starter data

In [169]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
from make_standings import Standings
from make_charts import make_playoff_charts

In [110]:
df = pd.read_csv("schedules/schedule21.csv")
pr = pd.read_csv("data/pr_both.csv",squeeze=True,index_col="Side")
teams = sorted(list(set(df["team_home"])))

The pandas DataFrame df contains the 2021 regular season schedule.  Originally extracted from from the dataset [NFL scores and betting data](https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data?select=spreadspoke_scores.csv) on Kaggle.  The columns pr_score_home and pr_score_away are going to contain our projected average score for the home team and the away team.

In [111]:
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff
0,9/9/2021,2021,1,TB,NaN,NaN,DAL,NaN,NaN,False,True,False
1,9/12/2021,2021,1,ATL,NaN,NaN,PHI,NaN,NaN,False,True,False
2,9/12/2021,2021,1,BUF,NaN,NaN,PIT,NaN,NaN,False,True,False
3,9/12/2021,2021,1,CAR,NaN,NaN,NYJ,NaN,NaN,False,False,False
4,9/12/2021,2021,1,CIN,NaN,NaN,MIN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2022,2021,18,TB,NaN,NaN,CAR,NaN,NaN,True,True,False
268,1/9/2022,2021,18,ARI,NaN,NaN,SEA,NaN,NaN,True,True,False
269,1/9/2022,2021,18,DEN,NaN,NaN,KC,NaN,NaN,True,True,False
270,1/9/2022,2021,18,LVR,NaN,NaN,LAC,NaN,NaN,True,True,False


The pandas Series pr contains the offensive and defensive power ratings, as well as our value for home field advantage and for the average score of a team.

In [112]:
pr

Side
ARI_Off        1.262909
ATL_Off       -0.043689
BAL_Off        2.128331
BUF_Off        3.371342
CAR_Off       -1.474896
                ...    
TB_Def         1.214125
TEN_Def       -0.615743
WAS_Def       -0.022900
HFA            2.110000
mean_score    23.820000
Name: PR, Length: 66, dtype: float64

Here we use the offensive and defensive power ratings to fill in the projected scores.  For now, these projected scores will be the same used in the Simulator app.

In [113]:
df["pr_score_home"] = df.apply(lambda row: pr[row["team_home"]+"_Off"] - pr[row["team_away"]+"_Def"]  
                               + pr["HFA"]/2 + pr["mean_score"],axis=1)
df["pr_score_away"] = df.apply(lambda row: pr[row["team_away"]+"_Off"] - pr[row["team_home"]+"_Def"] 
                               - pr["HFA"]/2 + pr["mean_score"],axis=1)

In [114]:
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff
0,9/9/2021,2021,1,TB,NaN,NaN,DAL,29.644140,22.992638,False,True,False
1,9/12/2021,2021,1,ATL,NaN,NaN,PHI,26.253218,22.486983,False,True,False
2,9/12/2021,2021,1,BUF,NaN,NaN,PIT,27.358107,21.501640,False,True,False
3,9/12/2021,2021,1,CAR,NaN,NaN,NYJ,24.329178,20.426817,False,False,False
4,9/12/2021,2021,1,CIN,NaN,NaN,MIN,22.839442,24.468182,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2022,2021,18,TB,NaN,NaN,CAR,30.049023,20.075978,True,True,False
268,1/9/2022,2021,18,ARI,NaN,NaN,SEA,25.652134,24.199165,True,True,False
269,1/9/2022,2021,18,DEN,NaN,NaN,KC,21.869942,26.959271,True,True,False
270,1/9/2022,2021,18,LVR,NaN,NaN,LAC,24.991195,24.902873,True,True,False


## Finding bye weeks
We start by building a function to find the bye week of a given team.  We begin using Buffalo as an example, and later we write the general function.

In [115]:
# In which rows is Buffalo the home team?
df["team_home"] == "BUF"

0      False
1      False
2       True
3      False
4      False
       ...  
267    False
268    False
269    False
270    False
271    False
Name: team_home, Length: 272, dtype: bool

In [116]:
# How many?
(df["team_home"] == "BUF").sum()

9

In [117]:
# What about as the away team?
(df["team_away"] == "BUF").sum()

8

In [118]:
# As either?  The vertical line | represents "or" for pandas objects.
((df["team_home"] == "BUF") | (df["team_away"] == "BUF")).sum()

17

In [119]:
# Here are all the rows in which Buffalo appears.
df[(df["team_home"] == "BUF") | (df["team_away"] == "BUF")]

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff
2,9/12/2021,2021,1,BUF,NaN,NaN,PIT,27.358107,21.501640,False,True,False
22,9/19/2021,2021,2,MIA,NaN,NaN,BUF,22.986174,25.670034,True,True,False
33,9/26/2021,2021,3,BUF,NaN,NaN,WAS,28.269243,19.804333,False,False,False
50,10/3/2021,2021,4,BUF,NaN,NaN,HOU,31.728213,17.734382,False,True,False
78,10/10/2021,2021,5,KC,NaN,NaN,BUF,29.113000,24.808778,False,True,False
93,10/18/2021,2021,6,TEN,NaN,NaN,BUF,25.007508,26.752086,False,True,False
109,10/31/2021,2021,8,BUF,NaN,NaN,MIA,27.780034,20.876174,True,True,False
126,11/7/2021,2021,9,JAX,NaN,NaN,BUF,21.750005,27.857927,False,True,False
140,11/14/2021,2021,10,NYJ,NaN,NaN,BUF,20.496794,27.065416,True,True,False
151,11/21/2021,2021,11,BUF,NaN,NaN,IND,27.858805,23.472589,False,True,False


In [120]:
# Another way to get the same thing.
df[df.apply(lambda row: "BUF" in set(row), axis = 1)]

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff
2,9/12/2021,2021,1,BUF,NaN,NaN,PIT,27.358107,21.501640,False,True,False
22,9/19/2021,2021,2,MIA,NaN,NaN,BUF,22.986174,25.670034,True,True,False
33,9/26/2021,2021,3,BUF,NaN,NaN,WAS,28.269243,19.804333,False,False,False
50,10/3/2021,2021,4,BUF,NaN,NaN,HOU,31.728213,17.734382,False,True,False
78,10/10/2021,2021,5,KC,NaN,NaN,BUF,29.113000,24.808778,False,True,False
93,10/18/2021,2021,6,TEN,NaN,NaN,BUF,25.007508,26.752086,False,True,False
109,10/31/2021,2021,8,BUF,NaN,NaN,MIA,27.780034,20.876174,True,True,False
126,11/7/2021,2021,9,JAX,NaN,NaN,BUF,21.750005,27.857927,False,True,False
140,11/14/2021,2021,10,NYJ,NaN,NaN,BUF,20.496794,27.065416,True,True,False
151,11/21/2021,2021,11,BUF,NaN,NaN,IND,27.858805,23.472589,False,True,False


In [121]:
# Same thing, but asking only for the schedule_week entry, not for the whole row:
df["schedule_week"][(df["team_home"] == "BUF") | (df["team_away"] == "BUF")]

2       1
22      2
33      3
50      4
78      5
93      6
109     8
126     9
140    10
151    11
167    12
193    13
205    14
209    15
232    16
240    17
257    18
Name: schedule_week, dtype: int64

In [122]:
# or the following:
df.loc[(df["team_home"] == "BUF") | (df["team_away"] == "BUF"), "schedule_week"]

2       1
22      2
33      3
50      4
78      5
93      6
109     8
126     9
140    10
151    11
167    12
193    13
205    14
209    15
232    16
240    17
257    18
Name: schedule_week, dtype: int64

In [123]:
# or the following:
df["schedule_week"][df.apply(lambda row: "BUF" in set(row), axis = 1)]

2       1
22      2
33      3
50      4
78      5
93      6
109     8
126     9
140    10
151    11
167    12
193    13
205    14
209    15
232    16
240    17
257    18
Name: schedule_week, dtype: int64

What we really care about is, which week does not occur?

In [124]:
weeks_with_games = set(df["schedule_week"][(df["team_home"] == "BUF") | (df["team_away"] == "BUF")])
weeks_with_games

{1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [125]:
set(range(1,19))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [126]:
# Here is Buffalo's bye week:
set(range(1,19)).difference(weeks_with_games).pop()

7

We now turn that procedure into a general function.

In [127]:
def find_bye(team):
    weeks_with_games = set(df["schedule_week"][(df["team_home"] == team) | (df["team_away"] == team)])
    return set(range(1,19)).difference(weeks_with_games).pop()

Using dict comprehension to make a dictionary containing each team's bye week.

In [128]:
bye_dict = {t:find_bye(t) for t in teams}
bye_dict

{'ARI': 12,
 'ATL': 6,
 'BAL': 8,
 'BUF': 7,
 'CAR': 13,
 'CHI': 10,
 'CIN': 10,
 'CLE': 13,
 'DAL': 7,
 'DEN': 11,
 'DET': 9,
 'GB': 13,
 'HOU': 10,
 'IND': 14,
 'JAX': 7,
 'KC': 12,
 'LAC': 7,
 'LAR': 11,
 'LVR': 8,
 'MIA': 14,
 'MIN': 7,
 'NE': 14,
 'NO': 6,
 'NYG': 10,
 'NYJ': 6,
 'PHI': 14,
 'PIT': 7,
 'SEA': 9,
 'SF': 6,
 'TB': 9,
 'TEN': 13,
 'WAS': 9}

## Adjusting the predicted score for teams coming off a bye week.
We go through every row and check if the home team is coming off a bye.  Then we do the same thing for the away teams.  I'm not sure if there's a faster way to do this, but since it only has to be done once, no matter how many simulations we run, efficiency in this step doesn't matter too much.

In [129]:
# Initialize two new columns with False.
df["off_bye_home"] = False
df["off_bye_away"] = False

In [130]:
# The f"..{}" notation is an example of using f-strings, which are available as of Python 3.6.
for s in ["home","away"]:
    for i in df.index:
        if df.loc[i,"schedule_week"] == bye_dict[df.loc[i,f"team_{s}"]]+1:
            df.loc[i,f"off_bye_{s}"] = True

In [131]:
# Using list comprehension to find all the teams with a bye in Week 7:
[i for i,j in bye_dict.items() if j == 7]

['BUF', 'DAL', 'JAX', 'LAC', 'MIN', 'PIT']

In [132]:
# Let's check if we did this right for the teams who have their bye in Week 7.
df.loc[df["schedule_week"]==8, ["schedule_week","team_home","team_away","pr_score_home","pr_score_away","off_bye_home","off_bye_away"]]

,schedule_week,team_home,team_away,pr_score_home,pr_score_away,off_bye_home,off_bye_away
107,8,ARI,GB,25.548758,23.315942,False,False
108,8,ATL,CAR,25.754561,22.932692,False,False
109,8,BUF,MIA,27.780034,20.876174,True,False
110,8,CHI,SF,19.865340,23.475426,False,False
111,8,CLE,PIT,25.242726,20.709202,False,True
112,8,IND,TEN,27.315105,23.626743,False,False
113,8,DET,PHI,23.017634,23.486208,False,False
114,8,HOU,LAR,18.700719,27.716067,False,False
115,8,NYJ,CIN,23.258143,21.861525,False,False
116,8,LAC,NE,24.634255,21.845311,True,False


If the home team is off a bye, then we add 0.5 to the predicted home score, and subtract 0.5 from the predicted away score.  If the away team is off a bye, then we add 0.5 to the predicted away score, and subtract 0.5 from the predicted home score.

In [133]:
# Be careful not to evaluate this cell multiple times.
df.loc[df["off_bye_home"],"pr_score_home"] += 0.5
df.loc[df["off_bye_home"],"pr_score_away"] -= 0.5
df.loc[df["off_bye_away"],"pr_score_away"] += 0.5
df.loc[df["off_bye_away"],"pr_score_home"] -= 0.5

Notice how the above projections have changed.

In [134]:
df.loc[df["schedule_week"]==8, ["schedule_week","team_home","team_away","pr_score_home","pr_score_away","off_bye_home","off_bye_away"]]

,schedule_week,team_home,team_away,pr_score_home,pr_score_away,off_bye_home,off_bye_away
107,8,ARI,GB,25.548758,23.315942,False,False
108,8,ATL,CAR,25.754561,22.932692,False,False
109,8,BUF,MIA,28.280034,20.376174,True,False
110,8,CHI,SF,19.865340,23.475426,False,False
111,8,CLE,PIT,24.742726,21.209202,False,True
112,8,IND,TEN,27.315105,23.626743,False,False
113,8,DET,PHI,23.017634,23.486208,False,False
114,8,HOU,LAR,18.700719,27.716067,False,False
115,8,NYJ,CIN,23.258143,21.861525,False,False
116,8,LAC,NE,25.134255,21.345311,True,False


## Now we can run the simulation using these new projected scores.

In [136]:
# Random number generator from NumPy
rng = default_rng()

In [145]:
# sample array:
A = np.array([[3,1,10],[2,-5,4]])
A

array([[ 3,  1, 10],
       [ 2, -5,  4]])

In [147]:
# Normal distribution.  A represents the means and 10 represents the standard deviation.
rng.normal(A,10)

array([[-13.07840707,  -6.23125253,   1.13428943],
       [ -6.98212628,   6.95150997,   5.69464514]])

In [150]:
# You can also use a different standard deviation in each position if you prefer.
rng.normal(A,[[10,0,2],[3,100,20]])

array([[  6.44525918,   1.        ,   5.61512833],
       [  2.77798145, 103.0209987 ,   9.58437592]])

In [157]:
df[["score_home","score_away"]] = rng.normal(df[["pr_score_home","pr_score_away"]],10)

In [158]:
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff,off_bye_home,off_bye_away
0,9/9/2021,2021,1,TB,17.598056,32.731332,DAL,29.644140,22.992638,False,True,False,False,False
1,9/12/2021,2021,1,ATL,24.969825,19.599195,PHI,26.253218,22.486983,False,True,False,False,False
2,9/12/2021,2021,1,BUF,43.774981,24.556387,PIT,27.358107,21.501640,False,True,False,False,False
3,9/12/2021,2021,1,CAR,14.586304,7.613547,NYJ,24.329178,20.426817,False,False,False,False,False
4,9/12/2021,2021,1,CIN,21.822938,16.245092,MIN,22.839442,24.468182,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2022,2021,18,TB,2.274523,23.161687,CAR,30.049023,20.075978,True,True,False,False,False
268,1/9/2022,2021,18,ARI,18.179764,10.784484,SEA,25.652134,24.199165,True,True,False,False,False
269,1/9/2022,2021,18,DEN,15.699928,27.122550,KC,21.869942,26.959271,True,True,False,False,False
270,1/9/2022,2021,18,LVR,27.500276,22.551438,LAC,24.991195,24.902873,True,True,False,False,False


In [159]:
# Round to the nearest integer.
df[["score_home","score_away"]] = df[["score_home","score_away"]].round()
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff,off_bye_home,off_bye_away
0,9/9/2021,2021,1,TB,18.0,33.0,DAL,29.644140,22.992638,False,True,False,False,False
1,9/12/2021,2021,1,ATL,25.0,20.0,PHI,26.253218,22.486983,False,True,False,False,False
2,9/12/2021,2021,1,BUF,44.0,25.0,PIT,27.358107,21.501640,False,True,False,False,False
3,9/12/2021,2021,1,CAR,15.0,8.0,NYJ,24.329178,20.426817,False,False,False,False,False
4,9/12/2021,2021,1,CIN,22.0,16.0,MIN,22.839442,24.468182,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2022,2021,18,TB,2.0,23.0,CAR,30.049023,20.075978,True,True,False,False,False
268,1/9/2022,2021,18,ARI,18.0,11.0,SEA,25.652134,24.199165,True,True,False,False,False
269,1/9/2022,2021,18,DEN,16.0,27.0,KC,21.869942,26.959271,True,True,False,False,False
270,1/9/2022,2021,18,LVR,28.0,23.0,LAC,24.991195,24.902873,True,True,False,False,False


In [160]:
# With our current set-up, there can be negative scores.
df[["score_home","score_away"]].min()

score_home    -6.0
score_away   -13.0
dtype: float64

In [162]:
# So we remove those.
df[["score_home","score_away"]] = df[["score_home","score_away"]].mask(df[["score_home","score_away"]] < 0, 0)
df[["score_home","score_away"]].min()

score_home    0.0
score_away    0.0
dtype: float64

In [164]:
# By default with this method, there are too many ties.
(df["score_home"] == df["score_away"]).sum()

6

In [165]:
# So we write some ad hoc code to get rid of most ties.  It slightly favors the home team.
def adjust_ties(df):
    tied_games = df.loc[df.score_home == df.score_away,["score_home","score_away"]].copy()
    x = rng.normal(size=len(tied_games))
    tied_games.iloc[np.where((x > -1.6) & (x < 0))[0]] += (0,3)
    tied_games.iloc[np.where(x >= 0)[0]] += (3,0)
    df.loc[tied_games.index,["score_home","score_away"]] = tied_games
    return df

In [166]:
df = adjust_ties(df)

In [167]:
# Most or all of the ties will now have disappeared.
(df["score_home"] == df["score_away"]).sum()

0

## Putting the whole season simulation method into a single function.

In [177]:
def simulate_season(df):
    df[["score_home","score_away"]] = rng.normal(df[["pr_score_home","pr_score_away"]],10)
    df[["score_home","score_away"]] = df[["score_home","score_away"]].round()
    df[["score_home","score_away"]] = df[["score_home","score_away"]].mask(df[["score_home","score_away"]] < 0, 0)
    df = adjust_ties(df)
    return df

In [178]:
simulate_season(df)

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff,off_bye_home,off_bye_away
0,9/9/2021,2021,1,TB,31.0,0.0,DAL,29.644140,22.992638,False,True,False,False,False
1,9/12/2021,2021,1,ATL,28.0,36.0,PHI,26.253218,22.486983,False,True,False,False,False
2,9/12/2021,2021,1,BUF,37.0,8.0,PIT,27.358107,21.501640,False,True,False,False,False
3,9/12/2021,2021,1,CAR,18.0,10.0,NYJ,24.329178,20.426817,False,False,False,False,False
4,9/12/2021,2021,1,CIN,22.0,33.0,MIN,22.839442,24.468182,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2022,2021,18,TB,23.0,15.0,CAR,30.049023,20.075978,True,True,False,False,False
268,1/9/2022,2021,18,ARI,37.0,16.0,SEA,25.652134,24.199165,True,True,False,False,False
269,1/9/2022,2021,18,DEN,38.0,31.0,KC,21.869942,26.959271,True,True,False,False,False
270,1/9/2022,2021,18,LVR,54.0,0.0,LAC,24.991195,24.902873,True,True,False,False,False


In [179]:
# If we run it again, we'll get different results.
simulate_season(df)

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_score_home,pr_score_away,in_div,in_conf,schedule_playoff,off_bye_home,off_bye_away
0,9/9/2021,2021,1,TB,21.0,26.0,DAL,29.644140,22.992638,False,True,False,False,False
1,9/12/2021,2021,1,ATL,15.0,43.0,PHI,26.253218,22.486983,False,True,False,False,False
2,9/12/2021,2021,1,BUF,35.0,3.0,PIT,27.358107,21.501640,False,True,False,False,False
3,9/12/2021,2021,1,CAR,42.0,24.0,NYJ,24.329178,20.426817,False,False,False,False,False
4,9/12/2021,2021,1,CIN,31.0,11.0,MIN,22.839442,24.468182,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2022,2021,18,TB,30.0,27.0,CAR,30.049023,20.075978,True,True,False,False,False
268,1/9/2022,2021,18,ARI,13.0,26.0,SEA,25.652134,24.199165,True,True,False,False,False
269,1/9/2022,2021,18,DEN,18.0,42.0,KC,21.869942,26.959271,True,True,False,False,False
270,1/9/2022,2021,18,LVR,31.0,36.0,LAC,24.991195,24.902873,True,True,False,False,False


## Going from scores to standings and playoff seeds.
We imported a constructor called Standings from the make_standings.py file.  Here is an example of how it works.

These standings are based on the scores that we just constructed.

In [170]:
stand = Standings(df)

In [171]:
# Playoff seeds, in order from 1-7.
stand.playoffs

{'AFC': ['BAL', 'KC', 'BUF', 'IND', 'LVR', 'CLE', 'LAC'],
 'NFC': ['SF', 'DET', 'TB', 'WAS', 'NO', 'GB', 'ATL']}

In [192]:
# Example of getting the explicit seeds.
for j,team in enumerate(stand.playoffs["AFC"]):
    print(j+1, team)

1 BAL
2 KC
3 BUF
4 IND
5 LVR
6 CLE
7 LAC


In [173]:
# Division ranks, in order from 1-4.
stand.div_ranks

{'AFC East': ['BUF', 'MIA', 'NE', 'NYJ'],
 'AFC North': ['BAL', 'CLE', 'CIN', 'PIT'],
 'AFC South': ['IND', 'JAX', 'TEN', 'HOU'],
 'AFC West': ['KC', 'LVR', 'LAC', 'DEN'],
 'NFC East': ['WAS', 'DAL', 'PHI', 'NYG'],
 'NFC North': ['DET', 'GB', 'MIN', 'CHI'],
 'NFC South': ['TB', 'NO', 'ATL', 'CAR'],
 'NFC West': ['SF', 'ARI', 'SEA', 'LAR']}

In [176]:
# Overall standings.
stand.standings

,Team,Wins,Losses,Ties,Points_scored,Points_allowed,WLT,Division,Conference,Division_rank
BUF,BUF,11,6,0,467.0,381.0,0.647059,AFC East,AFC,1
MIA,MIA,7,10,0,393.0,363.0,0.411765,AFC East,AFC,2
NE,NE,7,10,0,370.0,400.0,0.411765,AFC East,AFC,3
NYJ,NYJ,6,11,0,335.0,412.0,0.352941,AFC East,AFC,4
BAL,BAL,11,6,0,394.0,307.0,0.647059,AFC North,AFC,1
CLE,CLE,10,7,0,440.0,379.0,0.588235,AFC North,AFC,2
CIN,CIN,8,9,0,399.0,399.0,0.470588,AFC North,AFC,3
PIT,PIT,7,10,0,415.0,476.0,0.411765,AFC North,AFC,4
IND,IND,10,7,0,435.0,325.0,0.588235,AFC South,AFC,1
JAX,JAX,8,9,0,412.0,460.0,0.470588,AFC South,AFC,2


## Making a chart with the playoff rankings
To use the make_playoff_charts function that we imported at the top, we need to have a dictionary keeping track of how often different teams are different seeds.

In [183]:
# Getting the teams in the different conferences.
div_series = pd.read_csv("data/divisions.csv",squeeze=True,index_col=0)
teams = div_series.index
conf_teams = {}
for conf in ["AFC","NFC"]:
    conf_teams[conf] = [t for t in teams if div_series[t][:3]==conf]
print(conf_teams)

{'AFC': ['BAL', 'BUF', 'CIN', 'CLE', 'DEN', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LVR', 'MIA', 'NE', 'NYJ', 'PIT', 'TEN'], 'NFC': ['ARI', 'ATL', 'CAR', 'CHI', 'DAL', 'DET', 'GB', 'LAR', 'MIN', 'NO', 'NYG', 'PHI', 'SEA', 'SF', 'TB', 'WAS']}


In [185]:
playoff_dict = {}
    
for conf in ["AFC","NFC"]:
    playoff_dict[conf] = {i:{t:0 for t in conf_teams[conf]} for i in range(1,8)}

In [187]:
# This will record, for example, how often each team was a #3 seed in the AFC.
playoff_dict["AFC"][3]

{'BAL': 0,
 'BUF': 0,
 'CIN': 0,
 'CLE': 0,
 'DEN': 0,
 'HOU': 0,
 'IND': 0,
 'JAX': 0,
 'KC': 0,
 'LAC': 0,
 'LVR': 0,
 'MIA': 0,
 'NE': 0,
 'NYJ': 0,
 'PIT': 0,
 'TEN': 0}

In [196]:
%%time
# We'll use 200 simulations.  This will probably take about one minute.
reps = 200
for i in range(reps):
    df = simulate_season(df)
    stand = Standings(df)
    p = stand.playoffs
    for conf in ["AFC","NFC"]:
        for j,t in enumerate(p[conf]):
            playoff_dict[conf][j+1][t] += 1

CPU times: user 24.9 s, sys: 44.7 ms, total: 24.9 s
Wall time: 24.9 s


The point of making that playoff_dict dictionary was so that it can be used as an input to the make_playoff_charts function.

In [198]:
make_playoff_charts(playoff_dict)

alt.HConcatChart(...)

This was one example of adjusting the methods from the [NFL 2021 Regular Season Simulator](https://share.streamlit.io/christopherdavisuci/nfl2021-simulation/main/nfl_sim_app.py).  The main difference here is that we added some value to the power ratings of teams coming off a bye. There are unlimited possible ways to adjust the code, and you're encouraged to try out your own ideas!

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5d52731c-1eda-4ef1-a775-5055e022d838' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>